In [1]:
import os
import pandas as pd

from src.visualization.charts import GlobalModelCharts
from src.config import Config

1. For comparison

In [2]:
config = Config()

df_co2 = pd.read_csv(os.path.join(config.predictions, 'results_co2.csv'))
df_co2_pc = pd.read_csv(os.path.join(config.predictions, 'results_co2_per_capita.csv'))

df_co2 = df_co2.iloc[:, :-1]
df_co2_pc = df_co2_pc.iloc[:, :-1]

df_co2['MAPE'] = df_co2['MAPE'].str.rstrip('%').astype(float) / 100.0
df_co2_pc['MAPE'] = df_co2_pc['MAPE'].str.rstrip('%').astype(float) / 100.0

metrics = ['MAE', 'RMSE', 'R2']
for col in metrics:
    df_co2[col] = df_co2[col].astype(str).str.replace(',', '').astype(float)
    df_co2_pc[col] = df_co2_pc[col].astype(str).str.replace(',', '').astype(float)

df_co2['variant'] = 'co2'
df_co2_pc['variant'] = 'co2_per_capita'

df_merged = pd.concat([df_co2, df_co2_pc], ignore_index=True)


2. co2

In [3]:
variant = 'co2'
combined_csv_path = os.path.join(config.predictions, f'combined_results_{variant}.csv')
combined_df = pd.read_csv(combined_csv_path)

output_dir = os.path.join(config.predictions, variant)
charts = GlobalModelCharts(combined_df, output_dir, variant)
charts.generate_country_individual_charts()
charts.generate_country_combined_chart()
charts.generate_heatmap_mape(df_merged, variant)
charts.generate_barplot_mape(df_merged, variant)
charts.generate_barplot_r2(df_merged, variant)
charts.generate_best_model_barplots(df_merged, variant)

3. co2_per_capita

In [4]:
variant = 'co2_per_capita'
combined_csv_path = os.path.join(config.predictions, f'combined_results_{variant}.csv')
combined_df = pd.read_csv(combined_csv_path)

output_dir = os.path.join(config.predictions, variant)
charts = GlobalModelCharts(combined_df, output_dir, variant)
charts.generate_country_individual_charts()
charts.generate_country_combined_chart()
charts.generate_heatmap_mape(df_merged, variant)
charts.generate_barplot_mape(df_merged, variant)
charts.generate_barplot_r2(df_merged, variant)
charts.generate_best_model_barplots(df_merged, variant)

4. co2 vs co2_per_capita

In [5]:
charts.generate_boxplot_metric(df_merged,'MAE',config.predictions)
charts.generate_boxplot_metric(df_merged,'MAPE',config.predictions)
charts.generate_boxplot_metric(df_merged,'RMSE',config.predictions)
charts.generate_boxplot_metric(df_merged,'R2',config.predictions)